In [ ]:
!pip install -q anthropic

In [ ]:
from pathlib import Path
from pprint import pprint
from pydantic import BaseModel
from typing import List

def print_message(message):
    print(f"Message id: {message.id}")
    print(f"Input: {message.usage.input_tokens}; Output: {message.usage.output_tokens}")
    print(f"Stop reason: {message.stop_reason}")
    pprint(message.content)

    print()
    thinking_content_elements = [x.thinking for x in message.content if x.type == 'thinking']
    if len(thinking_content_elements) > 0:
      print(f"{'-' * 20} [Thinking] {'-' * 20}")
      for line in thinking_content_elements:
          print(line)

    print()
    print(f"{'-' * 20} [Text] {'-' * 20}")
    text_content_elements = [x.text for x in message.content if x.type == 'text']
    for line in text_content_elements:
      print(line)

In [ ]:
prompts = {
    "business_naming": "Give me top 3 ideas for naming my business. It's a software company that innovates in audio recording. The names should be interesting and witty.",
    "essay_about_humanity": "Write a short essay (up to 5 sentences) about the history of mankind.",
    "describe_ai": "Describe AI as if it were an animal.",
    "studying_tips": "Give me 5 practical tips to stay focused while studying."
}

## Basic usage

In [ ]:
from google.colab import userdata
from anthropic import Anthropic

api_key = userdata.get('ANTHROPIC_API_KEY')
client = Anthropic(api_key=api_key)

In [ ]:
available_models = client.beta.models.list()

In [ ]:
pprint(available_models.data)

In [ ]:
simple_response = client.messages.create(
    model="claude-3-haiku-20240307",
    messages=[{ "role": "user", "content": prompts["essay_about_humanity"] }],
    max_tokens=200
)

In [ ]:
print_message(simple_response)

## Streaming

In [ ]:
with client.messages.stream(
    model="claude-3-haiku-20240307",
    messages=[{ "role": "user", "content": prompts["studying_tips"] }],
    max_tokens=250
) as stream:
    # NOTE: `text_stream` iterates over the text deltas only
    # for text in stream.text_stream:
    #     print(text, end="", flush=True)

    for event in stream:
      pprint(event)

    streamed_message = stream.get_final_message()

In [ ]:
print_message(streamed_message)

## Reasoning

`claude-3-haiku-20240307` does not support thinking.

The `budget_tokens` parameter must be greater than or equal to 1024.

In [ ]:
low_reasoning_message = client.messages.create(
    model="claude-haiku-4-5-20251001",
    messages=[{ "role": "user", "content": prompts["business_naming"] }],
    thinking={"type": "enabled", "budget_tokens": 1024},
    max_tokens=1280
)

In [ ]:
print_message(low_reasoning_message)

In [ ]:
high_reasoning_message = client.messages.create(
    model="claude-haiku-4-5-20251001",
    messages=[{ "role": "user", "content": prompts["business_naming"] }],
    thinking={"type": "enabled", "budget_tokens": 4096},
    max_tokens=4352
)

In [ ]:
print_message(high_reasoning_message)

## Temperature

In [ ]:
deterministic_message = client.messages.create(
    model="claude-3-haiku-20240307",
    messages=[{ "role": "user", "content": prompts["describe_ai"] }],
    max_tokens=200,
    temperature=0.1
)

In [ ]:
print_message(deterministic_message)

In [ ]:
creative_message = client.messages.create(
    model="claude-3-haiku-20240307",
    messages=[{ "role": "user", "content": prompts["describe_ai"] }],
    max_tokens=200,
    temperature=0.9
)

In [ ]:
print_message(creative_message)

## Multi-turn conversations

> **Anthropic:**
> _"The Messages API can be used for either single queries or stateless multi-turn conversations."_


In [ ]:
multi_turn_conversation = [{ "role": "user", "content": "Generate a short role-play in no more than 5 lines: manager and employee discussing performance issues." }]
first_turn_message = client.messages.create(
    model="claude-haiku-4-5-20251001",
    messages=multi_turn_conversation,
    system="The manager should talk like a pirate.",
    max_tokens=1024
)

multi_turn_conversation.append({ "role": "assistant", "content": first_turn_message.content })

In [ ]:
print_message(first_turn_message)

In [ ]:
multi_turn_conversation.append({ "role": "user", "content": "Keep the role-play going for another 5 lines, but introduce a new development: the employee's wife enters the room dramatically and tells him that she's pregnant, leaving him genuinely surpirsed but at the same time desperately hoping to save his job because his family depends on it."})
second_turn_message = client.messages.create(
    model="claude-haiku-4-5-20251001",
    messages=multi_turn_conversation,
    max_tokens=2048,
    thinking={"type": "enabled", "budget_tokens": 1024},
)

In [ ]:
print_message(second_turn_message)

## Vision

<img src="https://freerangestock.com/sample/88947/painter-working-in-studio.jpg" />


In [ ]:
image_analysis_message = client.messages.create(
    model="claude-3-haiku-20240307",
    system="You are an expert image analyst. Keep your answer concise and structured.",
    messages=[
        {
            "role": "user",
            "content": [
                { "type": "text", "text": "Analyze this image and return a one-sentence summary followed by 5 key visible objects." },
                { "type": "image", "source": { "type": "url", "url": "https://freerangestock.com/sample/88947/painter-working-in-studio.jpg" } }
            ]
        }
    ],
    max_tokens=1024
)

In [ ]:
print_message(image_analysis_message)

## Files API

In [ ]:
path = Path("/content/test_doc_01.pdf")
with path.open("rb") as file_reader:
    file_upload_response = client.beta.files.upload(
        file=("test_doc_01.pdf", file_reader, "application/pdf")
    )

In [ ]:
file_id = file_upload_response.id
print(f"Created file with id: \"{file_id}\"")

In [ ]:
# NOTE: You can delete the previously created file, if not needed.
# client.beta.files.delete(file_id)

## Structured output

In [ ]:
class Classification(BaseModel):
    category: str
    confidence: float
    tags: List[str]
    sentiment: str


# NOTE: `claude-3-haiku-20240307` does not support PDF input.
structured_output_message = client.beta.messages.parse(
    model="claude-haiku-4-5-20251001",
    betas=["files-api-2025-04-14"],
    messages=[
        {
            "role": "user",
            "content": [
                { "type": "text", "text": "Classify this document:" },
                { "type": "document", "source": { "type": "file", "file_id": file_id } }
            ]
        }
    ],
    max_tokens=1024,
    output_format=Classification
)


In [ ]:
print_message(structured_output_message)

print()
print(f"Parsed output: {structured_output_message.parsed_output}")